In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Approach 2: binary classification using an RNN on fixed length data 

We use the annual data of each subject to predict whether his/her scores on `seven`, `bwcount` and `recall` goes up or down. 

But unlike in approach 1 where each sample corresponds to one subject and the number of waves varies from subject to subject, here we fix the number of waves to 4 for each sample. A sample is obtained by using a fixed-length sliding window over the waves of a subject. For example, if a subject has 6 waves, then we will use his/her 
* waves 1 to 4 to build the first sample, then 
* waves 2 to 5 to build the second sample, and finally 
* waves 3 to 6 to build the third sample. 



## Prepare the training data

In [2]:
import pickle
import numpy

### load the data 

In [3]:
[X, Y] = pickle.load(open('XY.pickle', 'br'))

In [4]:
Y[0]
Y[0][-1, 1]
Y[0][0, 1]

array([[0.3 , 0.8 , 1.  ],
       [0.6 , 0.8 , 1.  ],
       [0.35, 0.4 , 1.  ],
       [0.2 , 0.2 , 1.  ]])

0.2

0.8

In [5]:
i, j = numpy.ogrid[:3, :4]
x = 10*i + j
x

array([[ 0,  1,  2,  3],
       [10, 11, 12, 13],
       [20, 21, 22, 23]])

In [6]:
x.shape

(3, 4)

In [7]:
v = numpy.lib.stride_tricks.sliding_window_view(x, window_shape=(2, 4))
v

array([[[[ 0,  1,  2,  3],
         [10, 11, 12, 13]]],


       [[[10, 11, 12, 13],
         [20, 21, 22, 23]]]])

In [8]:
v[:, -1, :, :]

array([[[ 0,  1,  2,  3],
        [10, 11, 12, 13]],

       [[10, 11, 12, 13],
        [20, 21, 22, 23]]])

In [9]:
numpy.array([v[:, -1, :, :][0], v[:, -1, :, :][1]])

array([[[ 0,  1,  2,  3],
        [10, 11, 12, 13]],

       [[10, 11, 12, 13],
        [20, 21, 22, 23]]])

### functions to prepare the training data 

In [10]:
def slide_over_one_subject(subject_X, subject_Y, minimal_length = 4):
    """Create samples from the data of one subject using a sliding window

    subject_X, subject_Y: 2D numpy array, rows for waves and columns for features
    subject_new_X, subject_new_Y: 3D numpy array, axis 0 for sliding window, axis 1 for waves, axis 2 for features

    """
    subject_new_X, subject_new_Y = [], []
    num_features = subject_X.shape[1]
    subject_X_windowed = numpy.lib.stride_tricks.sliding_window_view(
        subject_X, 
        window_shape=(minimal_length, num_features)
    ) 
    subject_new_X = subject_X_windowed[:, -1, :, :]

    num_targets = subject_Y.shape[1]
    subject_Y_windowed = numpy.lib.stride_tricks.sliding_window_view(
        subject_Y, 
        window_shape=(minimal_length, num_targets)
    ) 
    subject_new_Y = subject_Y_windowed[:, -1, :, :]

    return subject_new_X, subject_new_Y

def sampling_fixed_length(X, Y, minimal_length = 4):
    new_X, new_Y = [], []
    for subject_X, subject_Y in zip(X, Y):
        if len(subject_Y)>= minimal_length:
            subject_new_X, subject_new_Y = slide_over_one_subject(subject_X, subject_Y, minimal_length=minimal_length)
            new_X += subject_new_X.tolist()
            new_Y += subject_new_Y.tolist()

    new_X, new_Y = numpy.array(new_X), numpy.array(new_Y)

    return new_X, new_Y 

# test 
_, new_Y = sampling_fixed_length(X[:2], Y[:2]) 
print (Y[:2])
print (new_Y)
print (new_Y[:, [0, -1], 1])
numpy.heaviside(new_Y[:, -1, 1]-new_Y[:, 0, 1], 0 )

[array([[0.3 , 0.8 , 1.  ],
        [0.6 , 0.8 , 1.  ],
        [0.35, 0.4 , 1.  ],
        [0.2 , 0.2 , 1.  ]]) array([[0.55, 1.  , 1.  ],
                                    [0.5 , 0.8 , 1.  ],
                                    [0.5 , 1.  , 1.  ],
                                    [0.4 , 0.8 , 1.  ],
                                    [0.5 , 1.  , 1.  ]])]
[[[0.3  0.8  1.  ]
  [0.6  0.8  1.  ]
  [0.35 0.4  1.  ]
  [0.2  0.2  1.  ]]

 [[0.55 1.   1.  ]
  [0.5  0.8  1.  ]
  [0.5  1.   1.  ]
  [0.4  0.8  1.  ]]

 [[0.5  0.8  1.  ]
  [0.5  1.   1.  ]
  [0.4  0.8  1.  ]
  [0.5  1.   1.  ]]]
[[0.8 0.2]
 [1.  0.8]
 [0.8 1. ]]


array([0., 0., 1.])

In [11]:
def generate_labels(Y, target_index: int):
    """Extract the training label for a particular target dimension 

    Y: 3D ndarray, axis 0 is sample, axis 1 is wave, axis 2 is score 
    """
    y_of_interest = Y[:, :, target_index] # 2D array 
    begin_score = Y[:, 0, target_index]
    end_score = Y[:, -1, target_index]

    labels = end_score - begin_score
    labels = numpy.heaviside(labels, 0)

    return labels.reshape((-1,1)) # 2d arrary, Nx1

# top level function for training data preparation 
# need to rerun for each target index rangeing from 0 to 2 
def prepare_training_data(X, Y, target_index: int, minimal_length=4):
    new_X, new_Y = sampling_fixed_length(X, Y, minimal_length=minimal_length) 
    new_Y = generate_labels(new_Y, target_index=target_index)
    return new_X, new_Y 


### Call actual functions to build the training set 

In [12]:
train_X, train_y = prepare_training_data(X, Y, target_index=2)

In [13]:
new_Y.shape

(3, 4, 3)

## The networks (RNN, CNN, FC)

In [14]:
import torch.nn as nn
import torch

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
      super().__init__()
      self.rnn = torch.nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    #   self.fc1 = torch.nn.Linear(hidden_size, 3)
      self.fc1 = torch.nn.Linear(hidden_size, 1)

  def forward(self, x):
      output, hn = self.rnn(x)
      x = self.fc1(hn[0])
    #   x = self.fc2(x)
      x = torch.sigmoid(x)
      return x

class FC(nn.Module):
  def __init__(self, input_size):
      super().__init__()
      self.fc1 = torch.nn.Linear(input_size, 50)
      self.fc2 = torch.nn.Linear(50, 20)
      self.fc3 = torch.nn.Linear(20, 10)
      self.fc4 = torch.nn.Linear(10, 1)

  def forward(self, x):
      x = torch.flatten(x, start_dim=1) # first dimension is batch 
      x = self.fc1(x)
      x = torch.sigmoid(x)
      x = self.fc2(x)
      x = torch.sigmoid(x)
      x = self.fc3(x)
      x = torch.sigmoid(x)
      x = self.fc4(x)
      x = torch.sigmoid(x)
    
      return x

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# test 
# PyTorch uses float32 while numpy by default uses float64. 
net = RNN(23, 5, 2)
net(torch.from_numpy(new_X.astype("float32"))[10:20]) 

NameError: name 'new_X' is not defined

## The trainer

In [175]:
def train(net, X, y, learning_rate, momentum):
    """

    net: a torch.nn instance
    X: 3D PyTorch Tensor, [sample, wave, feature]
    y: 2D PyTorch Tensor, Nx1, [sample, binary label]
    """
    import torch.optim as optim

    criterion = nn.BCELoss()
    # criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    batch_size = 32
    print_batch = batch_size*100

    loss_log = [] 

    for epoch in range(3):  # loop over the dataset multiple times

        running_loss = 0.0
        for i in range(0, X.shape[0], batch_size):

            optimizer.zero_grad()

            batch_X, batch_y = X[i:i+batch_size], y[i: i+batch_size]

            # forward + backward + optimize
            prediction = net.forward(batch_X)

            loss = criterion(prediction, batch_y)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % print_batch == 0 and i / print_batch > 1:   
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / print_batch:.7f}')
                loss_log.append(running_loss / print_batch)
                running_loss = 0.0
                
    return loss_log

## Results for `bw_count` (target_index=2)

In [139]:
train_X, train_y = prepare_training_data(X, Y, target_index=2)
train_X = torch.from_numpy(train_X.astype("float32"))
train_y = torch.from_numpy(train_y.astype("float32"))



In [ ]:
train_X = torch.from_numpy(train_X).float()
train_y = torch.from_numpy(train_y).float()

In [ ]:
net = RNN(23, 5, 2).float()
net(torch.from_numpy(train_X.float())) 

### Using RNN

In [177]:
net = RNN(23, 5, 2)
loss_log_005_09 = train(net, train_X, train_y, 0.001, 0.9)

[1,  6401] loss: 0.0410919
[1,  9601] loss: 0.0206670
[1, 12801] loss: 0.0203088
[1, 16001] loss: 0.0211965
[1, 19201] loss: 0.0207921
[1, 22401] loss: 0.0229284
[1, 25601] loss: 0.0223974
[1, 28801] loss: 0.0184102
[1, 32001] loss: 0.0219626
[1, 35201] loss: 0.0234412
[1, 38401] loss: 0.0231453
[1, 41601] loss: 0.0241504
[1, 44801] loss: 0.0237658
[2,  6401] loss: 0.0415263
[2,  9601] loss: 0.0219045
[2, 12801] loss: 0.0201613
[2, 16001] loss: 0.0217067
[2, 19201] loss: 0.0213759
[2, 22401] loss: 0.0227287
[2, 25601] loss: 0.0223976
[2, 28801] loss: 0.0184094
[2, 32001] loss: 0.0219687
[2, 35201] loss: 0.0234349
[2, 38401] loss: 0.0231698
[2, 41601] loss: 0.0242604
[2, 44801] loss: 0.0237616
[3,  6401] loss: 0.0413330
[3,  9601] loss: 0.0223558
[3, 12801] loss: 0.0208378
[3, 16001] loss: 0.0214638
[3, 19201] loss: 0.0213551
[3, 22401] loss: 0.0229291
[3, 25601] loss: 0.0223969
[3, 28801] loss: 0.0184087
[3, 32001] loss: 0.0219747
[3, 35201] loss: 0.0234323
[3, 38401] loss: 0.0231736
[

### Using FC-NN

In [178]:
net = FC(23*4)
loss_log_005_09 = train(net, train_X, train_y, 0.1, 0.1)

[1,  6401] loss: 0.0396276
[1,  9601] loss: 0.0195470
[1, 12801] loss: 0.0192475
[1, 16001] loss: 0.0198780
[1, 19201] loss: 0.0197586
[1, 22401] loss: 0.0199663
[1, 25601] loss: 0.0205692
[1, 28801] loss: 0.0170741
[1, 32001] loss: 0.0184604
[1, 35201] loss: 0.0209495
[1, 38401] loss: 0.0210797
[1, 41601] loss: 0.0214626
[1, 44801] loss: 0.0214244
[2,  6401] loss: 0.0394253
[2,  9601] loss: 0.0195459
[2, 12801] loss: 0.0192468
[2, 16001] loss: 0.0198774
[2, 19201] loss: 0.0197572
[2, 22401] loss: 0.0199659
[2, 25601] loss: 0.0205681
[2, 28801] loss: 0.0170777
[2, 32001] loss: 0.0184628
[2, 35201] loss: 0.0209489
[2, 38401] loss: 0.0210788
[2, 41601] loss: 0.0214616
[2, 44801] loss: 0.0214236
[3,  6401] loss: 0.0394285
[3,  9601] loss: 0.0195448
[3, 12801] loss: 0.0192460
[3, 16001] loss: 0.0198769
[3, 19201] loss: 0.0197558
[3, 22401] loss: 0.0199656
[3, 25601] loss: 0.0205671
[3, 28801] loss: 0.0170814
[3, 32001] loss: 0.0184652
[3, 35201] loss: 0.0209484
[3, 38401] loss: 0.0210780
[

## Results for `seven` (target_index=1)


In [159]:
train_X, train_y = prepare_training_data(X, Y, target_index=1)
train_X = torch.from_numpy(train_X.astype("float32"))
train_y = torch.from_numpy(train_y.astype("float32"))

### Using RNN

In [160]:
net = RNN(23, 5, 2)
loss_log_005_09 = train(net, train_X, train_y, 0.05, 0.5)

[1,  6401] loss: 0.034
[1,  9601] loss: 0.017
[1, 12801] loss: 0.017
[1, 16001] loss: 0.017
[1, 19201] loss: 0.016
[1, 22401] loss: 0.017
[1, 25601] loss: 0.017
[1, 28801] loss: 0.016
[1, 32001] loss: 0.016
[1, 35201] loss: 0.018
[1, 38401] loss: 0.017
[1, 41601] loss: 0.018
[1, 44801] loss: 0.019
[2,  6401] loss: 0.034
[2,  9601] loss: 0.017
[2, 12801] loss: 0.017
[2, 16001] loss: 0.017
[2, 19201] loss: 0.016
[2, 22401] loss: 0.017
[2, 25601] loss: 0.017
[2, 28801] loss: 0.016
[2, 32001] loss: 0.016
[2, 35201] loss: 0.018
[2, 38401] loss: 0.017
[2, 41601] loss: 0.018
[2, 44801] loss: 0.019
[3,  6401] loss: 0.034
[3,  9601] loss: 0.017
[3, 12801] loss: 0.017
[3, 16001] loss: 0.017
[3, 19201] loss: 0.016
[3, 22401] loss: 0.017
[3, 25601] loss: 0.017
[3, 28801] loss: 0.016
[3, 32001] loss: 0.016
[3, 35201] loss: 0.017
[3, 38401] loss: 0.017
[3, 41601] loss: 0.018
[3, 44801] loss: 0.019


### Using FC-NN

In [164]:
net = FC(23*4)
loss_log_005_09 = train(net, train_X, train_y, 0.001, 0.1)

[1,  6401] loss: 0.038
[1,  9601] loss: 0.018
[1, 12801] loss: 0.018
[1, 16001] loss: 0.018
[1, 19201] loss: 0.018
[1, 22401] loss: 0.018
[1, 25601] loss: 0.018
[1, 28801] loss: 0.017
[1, 32001] loss: 0.017
[1, 35201] loss: 0.018
[1, 38401] loss: 0.017
[1, 41601] loss: 0.018
[1, 44801] loss: 0.019
[2,  6401] loss: 0.034
[2,  9601] loss: 0.017
[2, 12801] loss: 0.017
[2, 16001] loss: 0.017
[2, 19201] loss: 0.017
[2, 22401] loss: 0.017
[2, 25601] loss: 0.017
[2, 28801] loss: 0.016
[2, 32001] loss: 0.016
[2, 35201] loss: 0.018
[2, 38401] loss: 0.017
[2, 41601] loss: 0.018
[2, 44801] loss: 0.019
[3,  6401] loss: 0.034
[3,  9601] loss: 0.017
[3, 12801] loss: 0.017
[3, 16001] loss: 0.017
[3, 19201] loss: 0.016
[3, 22401] loss: 0.017
[3, 25601] loss: 0.017
[3, 28801] loss: 0.016
[3, 32001] loss: 0.016
[3, 35201] loss: 0.018
[3, 38401] loss: 0.017
[3, 41601] loss: 0.019
[3, 44801] loss: 0.019


## Results on `recall` (target_index=0)


In [166]:
train_X, train_y = prepare_training_data(X, Y, target_index=0)
train_X = torch.from_numpy(train_X.astype("float32"))
train_y = torch.from_numpy(train_y.astype("float32"))

### Using RNN

In [167]:
net = RNN(23, 5, 2)
loss_log_005_09 = train(net, train_X, train_y, 0.05, 0.9)

[1,  6401] loss: 0.040
[1,  9601] loss: 0.020
[1, 12801] loss: 0.019
[1, 16001] loss: 0.020
[1, 19201] loss: 0.020
[1, 22401] loss: 0.020
[1, 25601] loss: 0.021
[1, 28801] loss: 0.017
[1, 32001] loss: 0.018
[1, 35201] loss: 0.021
[1, 38401] loss: 0.021
[1, 41601] loss: 0.021
[1, 44801] loss: 0.021
[2,  6401] loss: 0.039
[2,  9601] loss: 0.019
[2, 12801] loss: 0.019
[2, 16001] loss: 0.020
[2, 19201] loss: 0.020
[2, 22401] loss: 0.020
[2, 25601] loss: 0.021
[2, 28801] loss: 0.017
[2, 32001] loss: 0.018
[2, 35201] loss: 0.021
[2, 38401] loss: 0.021
[2, 41601] loss: 0.021
[2, 44801] loss: 0.021
[3,  6401] loss: 0.039
[3,  9601] loss: 0.019
[3, 12801] loss: 0.019
[3, 16001] loss: 0.020
[3, 19201] loss: 0.020
[3, 22401] loss: 0.020
[3, 25601] loss: 0.021
[3, 28801] loss: 0.017
[3, 32001] loss: 0.018
[3, 35201] loss: 0.021
[3, 38401] loss: 0.021
[3, 41601] loss: 0.021
[3, 44801] loss: 0.021


### Using FC-NN

In [169]:
net = FC(23*4)
loss_log_005_09 = train(net, train_X, train_y, 0.01, 0.1)

[1,  6401] loss: 0.041
[1,  9601] loss: 0.020
[1, 12801] loss: 0.019
[1, 16001] loss: 0.020
[1, 19201] loss: 0.020
[1, 22401] loss: 0.020
[1, 25601] loss: 0.021
[1, 28801] loss: 0.018
[1, 32001] loss: 0.019
[1, 35201] loss: 0.021
[1, 38401] loss: 0.021
[1, 41601] loss: 0.022
[1, 44801] loss: 0.021
[2,  6401] loss: 0.040
[2,  9601] loss: 0.020
[2, 12801] loss: 0.019
[2, 16001] loss: 0.020
[2, 19201] loss: 0.020
[2, 22401] loss: 0.020
[2, 25601] loss: 0.021
[2, 28801] loss: 0.018
[2, 32001] loss: 0.019
[2, 35201] loss: 0.021
[2, 38401] loss: 0.021
[2, 41601] loss: 0.022
[2, 44801] loss: 0.021
[3,  6401] loss: 0.040
[3,  9601] loss: 0.020
[3, 12801] loss: 0.019
[3, 16001] loss: 0.020
[3, 19201] loss: 0.020
[3, 22401] loss: 0.020
[3, 25601] loss: 0.021
[3, 28801] loss: 0.018
[3, 32001] loss: 0.019
[3, 35201] loss: 0.021
[3, 38401] loss: 0.021
[3, 41601] loss: 0.022
[3, 44801] loss: 0.021


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=38ce621b-4696-4047-8b25-0501b493ce55' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>